In [32]:
import torch
from llm_toolkit.llm import LLM

llm = LLM("unsloth/Llama-3.2-1B-Instruct")
inputs = llm.tokenizer("This is my test, yes, my very test.", return_tensors="pt")
print(f"Inputs: {inputs['input_ids'].shape}")
with torch.no_grad():
    outputs = llm.model.generate(inputs.input_ids, 
                                    output_hidden_states=True, 
                                    return_dict_in_generate=True, 
                                    max_new_tokens=10, 
                                    min_new_tokens=10, 
                                    pad_token_id=llm.tokenizer.pad_token_id)
    

print("Number of output tokens?: ", len(outputs['hidden_states']))
print("Number of layers: ", len(outputs['hidden_states'][0]))
print("Number of layers: ", len(outputs['hidden_states'][1]))
print("Number of layers: ", len(outputs['hidden_states'][2]))
print("First token: ", outputs['hidden_states'][0][5].shape) # Layer 5 - contains all the tokens from input?
print("Second token: ", outputs['hidden_states'][1][5].shape) # Layer 5
print("Third token: ", outputs['hidden_states'][2][5].shape) # Layer 5
print("===")
print("First token: ", outputs['hidden_states'][0][5][0,0,:])
print("Second token: ", outputs['hidden_states'][0][5][0,1,:])




Inputs: torch.Size([1, 12])
Number of output tokens?:  10
Number of layers:  17
Number of layers:  17
Number of layers:  17
First token:  torch.Size([1, 12, 2048])
Second token:  torch.Size([1, 1, 2048])
Third token:  torch.Size([1, 1, 2048])
===
First token:  tensor([ 0.1540, -0.5038,  1.6057,  ..., -0.2343,  1.3638, -0.1373])
Second token:  tensor([ 0.0608, -0.1365, -0.1327,  ...,  0.0287, -0.0818, -0.0287])


In [36]:
import numpy as np


num_layers = len(outputs['hidden_states'][0])  # Number of layers
sequence_length = outputs['hidden_states'][0][0].shape[1]  # Sequence length
embedding_dim = outputs['hidden_states'][0][0].shape[2]  # Embedding dimension

embeddings = np.zeros((num_layers, sequence_length, embedding_dim))

token_index_list: list[str] = []
# Iterate over each layer and token to extract embeddings
for token_index in range(sequence_length):
    for layer_index in range(num_layers):
        embeddings[layer_index, token_index, :] = outputs['hidden_states'][0][layer_index][0, token_index, :].detach().numpy()
        token_index_list.append(token_index)


array([[[ 2.68554688e-03,  3.08227539e-03, -6.80541992e-03, ...,
          1.07574463e-03,  8.20159912e-04,  1.54876709e-03],
        [-3.54003906e-03, -2.06298828e-02,  1.75781250e-02, ...,
         -2.13623047e-03, -1.91650391e-02, -1.58691406e-02],
        [ 1.03149414e-02,  1.96533203e-02,  2.19726562e-02, ...,
         -1.19018555e-02,  6.34765625e-03,  8.23974609e-03],
        ...,
        [-1.94549561e-03, -5.12695312e-03,  2.41699219e-02, ...,
         -3.14331055e-03, -4.02832031e-03, -2.20947266e-02],
        [ 3.06701660e-03,  1.08642578e-02,  3.05175781e-02, ...,
          4.39453125e-02,  2.70996094e-02, -1.95312500e-02],
        [-1.20239258e-02, -2.24609375e-02,  2.89306641e-02, ...,
         -2.63671875e-02, -2.02636719e-02,  7.44628906e-03]],

       [[ 2.01542228e-02, -4.58822101e-02,  5.11894748e-02, ...,
         -1.37398615e-02,  5.68491109e-02,  6.34422153e-03],
        [ 3.51454467e-02, -1.44961728e-02, -7.65948370e-02, ...,
          4.59279418e-02, -3.84887755e